In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws

import pandas as pd
import numpy as np
import math

import datetime as dt
from datetime import date, timedelta,datetime
import time 

from fyers_apiv3.FyersWebsocket import data_ws
import csv

import matplotlib.pyplot as plt
import mplfinance as mpf

import scipy

from  time import sleep
import threading
import os
import pyotp
import requests
import json
import pytz
from urllib.parse import parse_qs,urlparse
import base64

from IPython.display import clear_output

import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"
client_id='HTHFWXFS49-100'
secret_key = 'JOQVVTY9UJ'
FY_ID = "YB00632"  
TOTP_KEY = "3Y7PAJSPNHXEADGRDD726MLHJW43AJGH"  
PIN = "1964" 

grant_type = "authorization_code"                  
response_type = "code"                         
state = "sample"                                  

appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)


generateTokenUrl = appSession.generate_authcode()
generateTokenUrl


def getEncodedString(string):
    string = str(string)
    base64_bytes = base64.b64encode(string.encode("ascii"))
    return base64_bytes.decode("ascii")
  



URL_SEND_LOGIN_OTP="https://api-t2.fyers.in/vagator/v2/send_login_otp_v2"
res = requests.post(url=URL_SEND_LOGIN_OTP, json={"fy_id":getEncodedString(FY_ID),"app_id":"2"}).json()   

if datetime.now().second % 30 > 27 : sleep(5)
URL_VERIFY_OTP="https://api-t2.fyers.in/vagator/v2/verify_otp"
res2 = requests.post(url=URL_VERIFY_OTP, json= {"request_key":res["request_key"],"otp":pyotp.TOTP(TOTP_KEY).now()}).json()  


ses = requests.Session()
URL_VERIFY_OTP2="https://api-t2.fyers.in/vagator/v2/verify_pin_v2"
payload2 = {"request_key": res2["request_key"],"identity_type":"pin","identifier":getEncodedString(PIN)}
res3 = ses.post(url=URL_VERIFY_OTP2, json= payload2).json()  


ses.headers.update({
    'authorization': f"Bearer {res3['data']['access_token']}"
})


TOKENURL="https://api-t1.fyers.in/api/v3/token"
payload3 = {"fyers_id":FY_ID,
           "app_id":client_id[:-4],
           "redirect_uri":redirect_uri,
           "appType":"100","code_challenge":"",
           "state":"None","scope":"","nonce":"","response_type":"code","create_cookie":True}

res3 = ses.post(url=TOKENURL, json= payload3).json()  


url = res3['Url']
parsed = urlparse(url)
auth_code = parse_qs(parsed.query)['auth_code'][0]


grant_type = "authorization_code" 

response_type = "code"  

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YB00632',
  'name': 'BABAN MARUTI PAWAR',
  'image': None,
  'display_name': None,
  'pin_change_date': '21-12-2023 21:01:26',
  'email_id': 'babanpawar8975@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '8975963982',
  'totp': True,
  'pwd_to_expire': 90}}

In [3]:
sym = 'NSE:NIFTYBANK-INDEX'

In [4]:
csv_file_path = 'prev_open_date.csv'

with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    
    for row in csv_reader:
        cell_value = row[0]

start_date = datetime.strptime(cell_value, '%Y-%m-%d')
start_date = start_date.date()
end_date = date.today()
print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2024-01-04
End Date: 2024-01-07


In [5]:
csv_file_path = 'monthly_expiry.csv'

with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    
    for row in csv_reader:
        cell_value = row[0]

monthly_exp_date = datetime.strptime(cell_value, '%Y-%m-%d')
monthly_exp_date=monthly_exp_date.date()
tdate = date.today()
print("Monthly Expiry:", monthly_exp_date)

Monthly Expiry: 2024-01-24


In [6]:
csv_file_path = 'weekly_expiry.csv'

# Create an empty list to store values from column N
expiry = ""

# Open the CSV file
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object using DictReader
    csv_reader = csv.DictReader(file)

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Assuming 'N' is the header for the column
        # Append the value from column 'N' to the list
        expiry = (str(row['BN']))

# Now, 'column_n_values' contains all the values from column 'N'
print("BN Expiry:", expiry)

BN Expiry: NSE:BANKNIFTY24110


In [7]:
csv_file_path = 'today_lvls.csv'

# Create an empty list to store values from column N
lvl = []

# Open the CSV file
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object using DictReader
    csv_reader = csv.DictReader(file)

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Assuming 'N' is the header for the column
        # Append the value from column 'N' to the list
        lvl.append(float(row['BN']))

# Now, 'column_n_values' contains all the values from column 'N'
print("lvl:", lvl)

lvl: [48350.5, 48282.05, 48066.95, 47864.45, 47734.75, 47734.75, 47734.75]


In [8]:
historydata = pd.DataFrame()

In [9]:
def gethistorydata(st,ed,res):
    data = {
            "symbol":str(sym),
            "resolution":str(res),
            "date_format":"1",
            "range_from":st,
            "range_to":ed,
            "cont_flag":"0"
            }
    repo = fyers.history(data=data)
#     print(repo)
    data = pd.DataFrame.from_dict(repo['candles'])
    #global cols
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    
    data['datetime'] = pd.to_datetime(data['datetime'],unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    global historydata
#     data = data.set_index('datetime')
    historydata = data
    historydata['datetime'] = pd.to_datetime(historydata['datetime'])

    historydata['date'] = historydata['datetime'].dt.date
    historydata['time'] = historydata['datetime'].dt.time

    historydata.drop(columns=['datetime'], inplace=True)
    historydata.drop(columns=['volume'], inplace=True)
    
    #append(historydata,data)],axis=0)
    #historydata.to_csv("75op.csv")
    #print(historydata)

In [10]:
def pointchecker(pt,rg):
    point_value = pt
#     rg = 30
    for level in lvl:
        if abs(point_value - level) <= rg:
            return True
    else:
            return False

In [11]:
def directional_change(close: np.array, high: np.array, low: np.array, sigma: float):
    
    up_zig = True # Last extreme is a bottom. Next is a top. 
    tmp_max = high[0]
    tmp_min = low[0]
    tmp_max_i = 0
    tmp_min_i = 0

    tops = []
    bottoms = []

    for i in range(len(close)):
        if up_zig: # Last extreme is a bottom
            if high[i] > tmp_max:
                # New high, update 
                tmp_max = high[i]
                tmp_max_i = i
            elif close[i] < tmp_max - tmp_max * sigma: 
                # Price retraced by sigma %. Top confirmed, record it
                # top[0] = confirmation index
                # top[1] = index of top
                # top[2] = price of top
                top = [i, tmp_max_i, tmp_max]
                tops.append(top)

                # Setup for next bottom
                up_zig = False
                tmp_min = low[i]
                tmp_min_i = i
        else: # Last extreme is a top
            if low[i] < tmp_min:
                # New low, update 
                tmp_min = low[i]
                tmp_min_i = i
            elif close[i] > tmp_min + tmp_min * sigma: 
                # Price retraced by sigma %. Bottom confirmed, record it
                # bottom[0] = confirmation index
                # bottom[1] = index of bottom
                # bottom[2] = price of bottom
                bottom = [i, tmp_min_i, tmp_min]
                bottoms.append(bottom)

                # Setup for next top
                up_zig = True
                tmp_max = high[i]
                tmp_max_i = i

    return tops, bottoms

In [12]:
def get_extremes(ohlc: pd.DataFrame, sigma: float):
    tops, bottoms = directional_change(ohlc['close'], ohlc['high'], ohlc['low'], sigma)
    tops = pd.DataFrame(tops, columns=['conf_i', 'ext_i', 'ext_p'])
    bottoms = pd.DataFrame(bottoms, columns=['conf_i', 'ext_i', 'ext_p'])
    tops['type'] = 1
    bottoms['type'] = -1
    extremes = pd.concat([tops, bottoms])
    extremes = extremes.set_index('conf_i')
    extremes = extremes.sort_index()
    return extremes

In [13]:
def plot_tolerance_range(level, arr, tolerance=30, color='lightblue', alpha=0.3):
    lower_bound = level - tolerance
    upper_bound = level + tolerance
    plt.fill_between(range(len(arr)), lower_bound, upper_bound, color=color, alpha=alpha,label='level range')
    plt.axhline(y=level, color='blue', linestyle='-', label=f'Level {level}')

In [14]:
neck=0
extra=0
midline=0
sl=0
target=0
rrr = 0

ready=0
count=0
status =0
tobs = 0
point1=0
point2=0
point3=0
sp=''
strike=""

In [15]:
current_time = time.localtime(time.time())
day_of_week = current_time.tm_wday

In [16]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
today = days[day_of_week]
print("Day of the week:", today)

Day of the week: Sunday


In [17]:
current_time = time.localtime(time.time())

# Extract year, month, and day
year = current_time.tm_year
month = current_time.tm_mon
day = current_time.tm_mday

# Print today's date
print("Today's date:", f"{year}-{month:02d}-{day:02d}")

Today's date: 2024-01-07


In [19]:
def onmessage(message):
#     clear_output(wait=True)
    print(message,"\ttime: ",time.localtime().tm_hour,time.localtime().tm_min,time.localtime().tm_sec)
    time.sleep(1)

    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)  
    global historydata,neck,extra,midline,sl,target,rrr,ready,count,status,tobs,point3,point2,point1,sp,strike

    for sigma in [0.001, 0.002, 0.0005]:
        #print(sigma)
        if (int(cmin) % 5 == 0 and int(csec)==1):
            gethistorydata(start_date,end_date,5)
            ext = get_extremes(historydata, sigma) 
#             print(ext,sigma)
        
            if(historydata['open'][ext['ext_i'].iloc[-3]] > historydata['close'][ext['ext_i'].iloc[-3]]):
                point3 = historydata['close'][ext['ext_i'].iloc[-3]] 
            else:
                point3 = historydata['open'][ext['ext_i'].iloc[-3]]

            point2 = ext['ext_p'].iloc[-2] ########

            if(historydata['open'][ext['ext_i'].iloc[-1]] > historydata['close'][ext['ext_i'].iloc[-1]]):
                point1 = historydata['close'][ext['ext_i'].iloc[-1]] 
            else:
                point1 = historydata['open'][ext['ext_i'].iloc[-1]]

            if(pointchecker(point2,30) == True and point1 > point3 and point1<point2):
                neck = historydata['high'][ext['ext_i'].iloc[-2]]
                extra = neck - (neck*0.001) 
                midline  = (neck + extra)/2 
                sl = neck
                target = point1 
                rrr = (midline-target)/(sl-midline)
                ready = 1
                print("WAITING TO TOUCH IN ",sigma)

            if __name__ == '__main__':
                historydata['date'] = historydata['date'].astype('datetime64[s]')
                historydata = historydata.set_index('date')
                
                mpf.plot(historydata, type='candle', style='yahoo', ylabel='Price',title=str(sigma), hlines=lvl) 

                tops, bottoms = directional_change(historydata['close'], historydata['high'], historydata['low'],sigma)

                plt.style.use('dark_background')
                plt.figure(figsize=(36, 22))
                plt.grid(True)
                pd.Series(historydata['close'].to_numpy()).plot(label='historydata')

                # Plotting the extreme points
                plt.scatter(ext.index, ext['ext_p'], label='extreme points', color='red')
                for i, txt in enumerate(ext['ext_p']):
                    plt.annotate(f'{txt:.2f}', (ext.index[i], txt), xytext=(10, 10), textcoords='offset points', ha='right', va='bottom', color='white', fontsize=12, arrowprops=dict(arrowstyle='->'))


                plt.plot(ext.index, ext['ext_p'], linestyle='-', color='orange', label='Connect extremes')
                point2_y = ext['ext_p'].iloc[-2]  # Get the y-value for point2
                plt.axhline(y=point2_y, color='green', linestyle='-')
                eline = ((historydata['low'][ext['ext_i'].iloc[-2]])*0.001) + historydata['low'][ext['ext_i'].iloc[-2]]
                plt.axhline(y=eline, color='green', linestyle='-')
                nk = historydata['low'][ext['ext_i'].iloc[-2]]
                md = (nk + eline)/2
                plt.axhline(y=md, color='pink', linestyle='-')

                plt.xlabel('no of candles')
                plt.ylabel('price')
                plt.legend()
                
                for level in lvl:  # Example levels, replace with your actual levels
                    plot_tolerance_range(level, historydata['close'].to_numpy(), tolerance=30, color='lightblue', alpha=0.3, )
                
                for level in lvl:
                    plt.text(plt.xlim()[0], level, f'Level {level}', ha='right', va='center', fontsize=12)

                plt.show()
                
#                 time.sleep(8)

    if(midline!=0 and status!=1 and message['ltp']>=midline and message['ltp']>point1 and ready == 1 and rrr>=3 and count<2):
        for i in lvl:
            if(i< mid and i> target):
                target = i+30
                rrr = (sl-midline)/(midline-target)
                tobs = 1
        
        if(rrr>=1.5 and tobs ==1):    
            entryprice = message['ltp']
            if(today=="Wednesday" or tdate==monthly_exp_date):
                sp =(int(math.floor(entryprice / 100.0)) * 100)-100
            else:
                sp =(int(math.floor(entryprice / 100.0)) * 100)
            strike = expiry + str(sp) + "CE"
            edata = {
            "symbol": str(strike),
            "qty":15,
            "type":2,
            "side":1,
            "productType":"INTRADAY",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
            eresponse = fyers.place_order(data=edata)
            print(eresponse)

            print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
            entryhour = time.localtime(time.time()).tm_hour
            entrymin =  time.localtime(time.time()).tm_min
            entrydate = date.today()
            count = count+1
            status = 1
            start_time = time.time()

        elif(tobs==0):
            entryprice = message['ltp']
            if(today=="Wednesday" or tdate==monthly_exp_date):
                sp =(int(math.floor(entryprice / 100.0)) * 100)-100
            else:
                sp =(int(math.floor(entryprice / 100.0)) * 100)
            strike = expiry + str(sp) + "CE"
            edata = {
            "symbol": str(strike),
            "qty":15,
            "type":2,
            "side":1,
            "productType":"INTRADAY",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
            eresponse = fyers.place_order(data=edata)
            print(eresponse)

            print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
            entryhour = time.localtime(time.time()).tm_hour
            entrymin =  time.localtime(time.time()).tm_min
            entrydate = date.today()
            count = count+1
            status = 1
            start_time = time.time()

        
        elif(target!=0 and sl!=0 and status==1 and (message['ltp']<=target or message['ltp']>=sl)):    # message['ltp']>=rlvl
                exit_pos = strike+"-INTRADAY"
                exdata = {"id":str(exit_pos)}
                exresponse = fyers.exit_positions(data=exdata)
                print(exresponse)


                exitprice = message['ltp']
                print("ENTRY EXITED FOR ",sigma,"ON ",exitprice)
                status = 0
                ready=0
                sl=0
                target=0


                with open('w_backtest_report.csv', 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    exitprice = message['ltp']
                    exithour = time.localtime(time.time()).tm_hour
                    exitmin =  time.localtime(time.time()).tm_min       #historydata.iloc[-1]['time']
                    exitdate = date.today()
                    pandl = exitprice-entryprice
                    new_row = [sym,entrydate,entryhour,entrymin,entryprice,exitdate,exithour,exitmin,exitprice,pandl]
                    writer.writerow(new_row)
                    print("EXPORTED TRADE DATA")

                if(message['symbol']==sym and count>=2):
                    data_type= "SymbolUpdate"
                    symbol_to_unsubscribe= [sym]
                    wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)


        elif(status==1):
            current_time = time.time()
            elapsed_minutes = int((current_time - start_time) / 60)
            remaining_minutes = target_minutes - elapsed_minutes

            if(remaining_minutes <= 0 ):
                exit_pos = strike+"-INTRADAY"
                exdata = {"id":str(exit_pos)}
                exresponse = fyers.exit_positions(data=exdata)
                print(exresponse)
                
                exitprice = message['ltp']
                print("ENTRY EXITED FOR ",sigma,"ON ",exitprice)
                status = 0
                ready=0
                sl=0
                target=0

                with open('w_backtest_report.csv', 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    exitprice = message['ltp']
                    exithour = time.localtime(time.time()).tm_hour
                    exitmin =  time.localtime(time.time()).tm_min       #historydata.iloc[-1]['time']
                    exitdate = date.today()
                    pandl = exitprice-entryprice
                    new_row = [symb,entrydate,entryhour,entrymin,entryprice,exitdate,exithour,exitmin,exitprice,pandl]
                    writer.writerow(new_row)
                    print("EXPORTED TRADE DATA")

                if(message['symbol']==sym and count>=2):
                    data_type= "SymbolUpdate"
                    symbol_to_unsubscribe= [sym]
                    wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)
                        
    if(status!=1 and time.localtime(time.time()).tm_hour>=14 and time.localtime(time.time()).tm_min>=30 and message['symbol']==sym):
        data_type= "SymbolUpdate"
        symbol_to_unsubscribe= [sym]
        wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)


def onerror(message):
    print("Error:", message)


def onclose(message):
    print("Connection closed:", message)


def onopen():
    data_type = "SymbolUpdate"

    symbols = [sym]
    wfyers.subscribe(symbols=symbols, data_type=data_type)

    wfyers.keep_running()


waccess_token = client_id+":"+access_token 

wfyers = data_ws.FyersDataSocket(
    access_token=waccess_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage            # Callback function to handle incoming messages from the WebSocket.
)

wfyers.connect()

{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'} 	time:  1 17 1
{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'} 	time:  1 17 2
{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'} 	time:  1 17 3
{'ltp': 48159.0, 'symbol': 'NSE:NIFTYBANK-INDEX', 'type': 'if'} 	time:  1 17 4


### --------------------------------------------------------------------------------------------------------------------------------------------------------------